In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import datetime as dt
# Any results you write to the current directory are saved as output.

In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
print('function created')

In [ ]:
inittime = dt.datetime.now()
print("Capturing train database...")
df_train = pd.read_csv("../input/train_V2.csv")
print("Train Database captured in ", dt.datetime.now()- inittime, " secs" )
df_train= reduce_mem_usage(df_train)

inittime = dt.datetime.now()
print("Capturing Test database...")
df_test = pd.read_csv("../input/test_V2.csv")
print("Test Database captured in ", dt.datetime.now()- inittime, " secs" )
df_test= reduce_mem_usage(df_test)

print("Train data set :\n",df_train.head())
print("Test data set :\n", df_test.head())

In [ ]:
df_train.info()

In [ ]:
df_train.loc[df_train['winPlacePerc'].isna()==True, 'winPlacePerc']= 0.5

In [ ]:
df_train['matchType'].value_counts()

In [ ]:
df_test['winPlacePerc']= 0.0
df_train['Type']= 'Train'
df_test['Type']= 'Test'
print(df_train.shape, df_test.shape)

In [ ]:
df= pd.concat([df_train, df_test], ignore_index=True)
del df_train
print(df.shape)

In [ ]:
df= reduce_mem_usage(df)

In [ ]:
df.columns

In [ ]:
# Counrtesy from kernel rejasupotaro/effective-feature-engineering
def min_by_team(df,df_Group, features):
    print("Working on Min by Team features...")
    inittime = dt.datetime.now()
    agg = df_Group[features].min()
    print("Features : Min Created in ", dt.datetime.now()- inittime)
    return df.merge(agg, suffixes=['', '_min'], how='left', on=['matchId', 'groupId'])


def max_by_team(df,df_Group, features):
    print("Working on Max by Team features...")
    inittime = dt.datetime.now()
    agg = df_Group[features].max()
    print("Features : MAX Created in ", dt.datetime.now()- inittime)
    return df.merge(agg, suffixes=['', '_max'], how='left', on=['matchId', 'groupId'])

def sum_by_team(df,df_Group, features):
    print("Working on Sum by Team features...")
    inittime = dt.datetime.now()
    agg = df_Group[features].sum()
    print("Features : SUM Created in ", dt.datetime.now()- inittime)
    return df.merge(agg, suffixes=['', '_sum'], how='left', on=['matchId', 'groupId'])

def median_by_team(df,df_Group, features):
    print("Working on Median by Team features...")
    inittime = dt.datetime.now()
    agg = df_Group[features].median()
    print("Features : MEDIAN Created in ", dt.datetime.now()- inittime)
    return df.merge(agg, suffixes=['', '_median'], how='left', on=['matchId', 'groupId'])

def mean_by_team(df,df_Group, features):
    print("Working on Mean by Team features...")
    inittime = dt.datetime.now()
    agg = df_Group[features].mean()
    print("Features : MEAN Created in ", dt.datetime.now()- inittime)
    return df.merge(agg, suffixes=['', '_mean'], how='left', on=['matchId', 'groupId'])

def rank_by_team(df,df_Group, features):
    print("Working on Rank by Team features...")
    inittime = dt.datetime.now()
    agg = df_Group[features].mean()
    print("Means Created")
    agg1 = agg.groupby('matchId')[features].rank(pct=True)
    print("Features : RANK Created in ", dt.datetime.now()- inittime)
    df= df.merge(agg, suffixes=['', '_mean'], how='left', on=['matchId', 'groupId'])
    return df.merge(agg1, suffixes=['', '_mean_rank'], how='left', on=['matchId', 'groupId'])
print("Functions created")

In [ ]:
#df= df.set_index(['Id', 'groupId', 'matchId','Type'])


Y_Column = 'winPlacePerc'

ColumnList = ['assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'maxPlace', 'numGroups', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints']


df['headshotKills_over_kills'] = df['headshotKills'] / df['kills']
df['headshotKills_over_kills'].fillna(0, inplace=True)
df['headshotKills_over_kills'].replace(np.inf, 0, inplace=True)

df['killPlace_over_maxPlace'] = df['killPlace'] / df['maxPlace']
df['killPlace_over_maxPlace'].fillna(0, inplace=True)
df['killPlace_over_maxPlace'].replace(np.inf, 0, inplace=True)

df['walkDistance_over_heals'] = df['walkDistance'] / df['heals']
df['walkDistance_over_heals'].fillna(0, inplace=True)
df['walkDistance_over_heals'].replace(np.inf, 0, inplace=True)

df['walkDistance_over_kills'] = df['walkDistance'] / df['kills']
df['walkDistance_over_kills'].fillna(0, inplace=True)
df['walkDistance_over_kills'].replace(np.inf, 0, inplace=True)

df= reduce_mem_usage(df)

cols_to_drop = ['Id', 'groupId', 'matchId', 'winPlacePerc','Type','matchType']
features = [col for col in ColumnList if col not in cols_to_drop]
df_Group = df.groupby(['matchId','groupId'])

#df= min_by_team(df,df_Group,features)
df= reduce_mem_usage(df)
#df= max_by_team(df,df_Group,features)
#df= reduce_mem_usage(df)
#df= sum_by_team(df,df_Group,features)
#df= reduce_mem_usage(df)
df= median_by_team(df,df_Group,features)
df= reduce_mem_usage(df)
#df= mean_by_team(df,df_Group,features)
df= rank_by_team(df,df_Group,features)
df= reduce_mem_usage(df)
print("New variables created")

for colname in ColumnList:
    df[colname+ '_1']= df[colname]/ (max(df[colname])- min(df[colname]))
    df= df.drop(columns= [colname])
    df= reduce_mem_usage(df)

    #df[colname]= df[colname]**0.5
    print(colname,'_1',' created')
df['totalDistance'] = df['rideDistance_1'] + df['walkDistance_1'] + df['swimDistance_1']
print("New variables created : totalDistance")

df['kills_assists'] = (df['kills_1'] + df['assists_1'])
print("New variables created : kills_assists")

df['healthitems'] = df['heals_1'] + df['boosts_1']
print("New variables created : healthitems")

df['kills_']= df['kills_1'] + df['longestKill_1']+ df['killStreaks_1']
print("New variables created : kills_")

df['killtypes']= df['headshotKills_1'] + df['roadKills_1']+ df['teamKills_1']+ df['vehicleDestroys_1']
print("New variables created : killtypes")

df['Others']= df['damageDealt_1'] + df['DBNOs_1']+ df['revives_1'] + df['weaponsAcquired_1']
print("New variables created : Others")

df['Points_'] = df['killPoints_1'] + df['winPoints_1']
print("New variables created : Points_")
df= reduce_mem_usage(df)
#df_Group1 = df.groupby(by = ['matchId'])
#df_Group2 = df.groupby(by = ['groupId'])

#def Create_Rank_Columns(db,dbGroup, NewColName, colName):
#    starttime= dt.datetime.now()
#    db[NewColName]= dbGroup[colName].rank(ascending= False)
#    print(NewColName, " created in time :", dt.datetime.now()-starttime )
#    return db

#df= Create_Rank_Columns(df,df_Group1,'Rank_Points1','Points_')
#df= Create_Rank_Columns(df,df_Group1,'Rank_kills1','kills_')
#df= Create_Rank_Columns(df,df_Group1,'Rank_killassist1','kills_assists')
#df= Create_Rank_Columns(df,df_Group1,'Rank_killtypes1','killtypes')
#df= Create_Rank_Columns(df,df_Group1,'Rank_healthitems1','healthitems')

#df= Create_Rank_Columns(df,df_Group2,'Rank_Points2','Points_')
#df= Create_Rank_Columns(df,df_Group2,'Rank_kills2','kills_')
#df= Create_Rank_Columns(df,df_Group2,'Rank_killassist2','kills_assists')
#df= Create_Rank_Columns(df,df_Group2,'Rank_killtypes2','killtypes')
#df= Create_Rank_Columns(df,df_Group2,'Rank_healthitems2','healthitems')

#df['Rank1']= (df['Rank_Points1']+ df['Rank_kills1'] + df['Rank_killassist1']+ df['Rank_killtypes1']+ df['Rank_healthitems1'])/5
#df['Rank2']= (df['Rank_Points2']+ df['Rank_kills2'] + df['Rank_killassist2']+ df['Rank_killtypes2']+ df['Rank_healthitems2'])/5

#df= df.drop(columns = ColumnList)
print(df.columns)

ColumnList = df.columns.tolist()
print(ColumnList)
ColumnList.remove('Id')
ColumnList.remove('groupId')
ColumnList.remove('matchId')
ColumnList.remove('Type')
ColumnList.remove('winPlacePerc')
ColumnList.remove('matchType')

df= reduce_mem_usage(df)

df_Train = df[df['Type']== 'Train']
df_Test = df[df['Type']== 'Test']
print("Train and Test Data sets created")

df_Train= df_Train.set_index(['Id', 'groupId', 'matchId','Type'])
df_Test= df_Test.set_index(['Id', 'groupId', 'matchId','Type'])
print("Indexes set")

X_Train = df_Train[ColumnList]
Y_Train = df_Train[Y_Column]

X_test = df_Test[ColumnList]

print(X_Train.shape, Y_Train.shape, X_test.shape)
del df, df_Train, df_Test

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

In [ ]:
X_Train.dtypes

In [ ]:
%%time
# For tuning parameters
starttime = dt.datetime.now()
d_train1 = lgb.Dataset(X_Train, label=Y_Train.values)
params = {}
params['learning_rate'] = 0.09
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['sub_feature'] = 0.8
params['num_leaves'] = 1000
params['min_data'] = 1
params['max_depth'] = 400
params['min_gain_to_split']= 0.0000001
clf1 = lgb.train(params, d_train1, 1000)
print("Model created in ", dt.datetime.now()- starttime)
#y_pred=clf1.predict(X_Train.head(1000))
y_pred=clf1.predict(X_test)

print(y_pred)
#[0.2522337  0.82816766 0.52467091 ... 0.40437544 0.70938478 0.22089762]
#CPU times: user 11 s, sys: 432 ms, total: 11.4 s
#Wall time: 3 s

In [ ]:
# Restore some columns
del X_Train
X_test = X_test.reset_index()
X_test["winPlacePerc"] = y_pred
X_test = X_test[['Id',"winPlacePerc"]]
df_test = df_test[['Id','groupId']]
df_test= df_test.merge(X_test, on = ['Id'])
del X_test
df_test["winPlacePerc"]= df_test["winPlacePerc"].clip(lower = 0.0, upper= 1.0)
print(df_test.head())

In [ ]:
df_test[['Id',"winPlacePerc"]].to_csv("submission.csv", index=False)